## 0. Import statements

In [1]:
import sbmlnetwork
import libsbml

import pandas as pd
import tellurium as te
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [2]:
rxns = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/Reactions_M1.csv')

## 1. Retrieve coordinates from KEGG
### 1a. Get the (x,y) coordinates for each species in the KEGG map

In [3]:
def get_kegg_orthology_ids(reaction_ids):
    orthology_dict = {}
    for reaction_id in reaction_ids:
        url = f"https://www.kegg.jp/dbget-bin/www_bget?rn:{reaction_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the orthology section
        orthology_section = soup.find('th', string="Orthology")
        if orthology_section:
            # Get the orthology ID from the adjacent <td> tag
            orthology_ids = orthology_section.find_next('td').find_all('a')
            # Filter IDs that start with 'K'
            orthology_dict[reaction_id] = [ortho_id.get_text() for ortho_id in orthology_ids if ortho_id.get_text().startswith('K')]
        else:
            orthology_dict[reaction_id] = []
    
    return orthology_dict

In [4]:
orthology_ids = get_kegg_orthology_ids(rxns['Reaction ID'])

In [5]:
import xml.etree.ElementTree as ET
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas

def map_reaction_to_entry_id(xml_file_path, reaction_orthology_dict):
    # Parse the XML file
    pathway = KGML_parser.read(open(xml_file_path, 'r'))

    # Initialize dictionary to hold the reaction ID -> entry ID mapping
    reaction_entry_map = {}

    # Loop through each entry in the pathway
    for entry in pathway.entries.values():
        # Check if the entry is of type 'ortholog'
        if entry.type == "ortholog":
            # Extract the orthology IDs (split by space, remove 'ko:' prefix)
            orthology_ids_in_entry = entry.name.replace('ko:', '').split()

            # Convert to set for comparison
            orthology_ids_in_entry_set = set(orthology_ids_in_entry)

            # Loop through the reaction dictionary
            for reaction_id, orthology_ids in reaction_orthology_dict.items():
                if len(orthology_ids)==0:
                    continue
                # Convert orthology_ids from the reaction dictionary to a set
                orthology_ids_set = set(orthology_ids)

                # Check if all orthology IDs in the entry are present in orthology_ids_set
                # CHANGING THIS AS AN EXPERIMENT: CHECK IF ANY OF THE ORTHOLOGY IDS IN THE ENTRY ARE PRESENT IN ORTHOLOGY_IDS_SET
                # if not orthology_ids_set.isdisjoint(orthology_ids_in_entry_set):
                # if orthology_ids_set.issubset(orthology_ids_in_entry_set):
                if not orthology_ids_set.isdisjoint(orthology_ids_in_entry_set):
                    # Add the mapping to the dictionary
                    if reaction_id not in reaction_entry_map:
                        reaction_entry_map[reaction_id] = []
                    reaction_entry_map[reaction_id].append(entry.id)  # Save the entry ID

    return reaction_entry_map

In [6]:
xml_file = 'ko01100.xml'
reaction_entry_map = map_reaction_to_entry_id(xml_file, orthology_ids)

In [7]:
pathway = KGML_parser.read(open(xml_file, 'r'))
reaction_dict = reaction_entry_map
    
# Initialize dictionary to hold the cpd ID and Reaction ID mapping
cpd_coordinates_map = {}

# Loop through the reaction dictionary
for reaction_id, entry_ids in reaction_dict.items():
    for entry_id in entry_ids:
        # Search for the reaction in the pathway
        reaction_entry = next((reaction for reaction in pathway.reactions if reaction.id == entry_id), None)

        if reaction_entry:
            newrns = []
            for rn in reaction_entry._names:
                newrns.append(rn.replace('rn:',''))
            if reaction_id in newrns:
                # Collect substrates
                sloop_counter = 0
                for substrate in reaction_entry._substrates:
                    substrate_entry = next((sub for sub in pathway.entries.values() if sub.id == substrate), None)
                    x, y = substrate_entry.graphics[0].x, substrate_entry.graphics[0].y
                    
                    rxnentry = str(reaction_entry)
                    # Split the string into lines and find the line containing 'Substrates'
                    substrates_line = next(line for line in rxnentry.splitlines() if 'Substrates:' in line)
                    # Extract the part of the line after 'Substrates:'
                    substrates_part = substrates_line.split('Substrates: ')[1]
                    # Split the substrates by comma and strip the 'cpd:' prefix
                    cpd_id = [substrate.strip().replace('cpd:', '') for substrate in substrates_part.split(',')][sloop_counter]
                    sloop_counter += 1

                    cpd_coordinates_map[f"{cpd_id}_{reaction_id}_{entry_id}"] = (x, y)

                # Collect products
                ploop_counter = 0
                for product in reaction_entry._products:
                    product_entry = next((prod for prod in pathway.entries.values() if prod.id == product), None)
                    x, y = product_entry.graphics[0].x, product_entry.graphics[0].y
                    
                    rxnentry = str(reaction_entry)
                    # Split the string into lines and find the line containing 'Substrates'
                    products_line = next(line for line in rxnentry.splitlines() if 'Products:' in line)
                    # Extract the part of the line after 'Substrates:'
                    products_part = products_line.split('Products: ')[1]
                    # Split the substrates by comma and strip the 'cpd:' prefix
                    cpd_id = [product.strip().replace('cpd:', '') for product in products_part.split(',')][ploop_counter]
                    ploop_counter += 1

                    cpd_coordinates_map[f"{cpd_id}_{reaction_id}_{entry_id}"] = (x, y)

In [8]:
data = pd.read_csv('/workspaces/ECFERS/kegg_labels_add.csv',dtype='str',encoding='us-ascii',encoding_errors='ignore')
data = data.where(data.notnull(), None)

In [9]:
# Create a new dictionary with replaced keys
cpd_coordinates_map_tran = {}
for key, value in cpd_coordinates_map.items():
    new_key = key
    # Check for each "KEGG ID" in the key string
    for _, row in data.iterrows():
        kegg_id = row['KEGG ID']
        id_value = row['ID']
        # Replace any occurrence of the "KEGG ID" within the key string
        if kegg_id in new_key:
            new_key = new_key.replace(kegg_id, id_value)
    # Add the modified key-value pair to the new dictionary
    cpd_coordinates_map_tran[new_key] = value

In [10]:
cpd_coordinates_map_tran

{'Glycine_R00945_7890': (3160.0, 1713.0),
 '_510CH2THF_R00945_7890': (2280.0, 2169.0),
 'THF_R00945_7890': (1936.0, 2049.0),
 'Serine_R00945_7890': (2965.0, 1713.0),
 'THF_R00945_5411': (4553.0, 791.0),
 '_510CH2THF_R00945_5411': (4788.0, 792.0),
 'Serine_R09099_1212': (2965.0, 1713.0),
 '_510MTHMPT_R09099_1212': (2057.0, 2696.0),
 'Pyr_R00209_1556': (2399.0, 1713.0),
 'AcCoA_R00209_1556': (2399.0, 1834.0),
 'Oxa_R00355_2421': (2234.0, 3105.0),
 'LAspartate_R00355_2421': (2284.0, 3105.0),
 'Oxa_R00355_4044': (2321.0, 1979.0),
 'Glu_R00355_4044': (3271.0, 2333.0),
 'LAspartate_R00355_4044': (2709.0, 2013.0),
 'Pyr_R00220_4482': (2399.0, 1713.0),
 'Serine_R00220_4482': (2965.0, 1713.0),
 'PEP_R00200_3083': (2399.0, 1614.0),
 'Pyr_R00200_3083': (2399.0, 1713.0),
 'dATP_R01138_1128': (3598.0, 377.0),
 'dADP_R01138_1128': (3684.0, 377.0),
 'dGDP_R01858_1116': (4160.0, 378.0),
 'dGTP_R01858_1116': (4257.0, 378.0),
 'PEP_R02320_3083': (2399.0, 1614.0),
 'Pyr_R02320_3083': (2399.0, 1713.0),
 '

## 2. Apply KEGG coordinates to SBMLNetwork layout

### 2a. Remove enzymes, inhibitors, and cofactors from the visual.

In [11]:
#remove any unused species
def remove_unused_species(sbml_file, output_file):
    # Read SBML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(sbml_file)
    model = document.getModel()
    
    if model is None:
        print("Failed to read model from SBML file.")
        return

    # Collect IDs of all species involved in reactions
    used_species = set()
    for reaction in model.getListOfReactions():
        for reactant in reaction.getListOfReactants():
            used_species.add(reactant.getSpecies())
        for product in reaction.getListOfProducts():
            used_species.add(product.getSpecies())
        for modifier in reaction.getListOfModifiers():
            used_species.add(modifier.getSpecies())
    
    # Collect IDs of all species in the model
    all_species = {species.getId() for species in model.getListOfSpecies()}
    
    # Determine unused species
    unused_species = all_species - used_species
    
    # Remove unused species
    for species_id in unused_species:
        species = model.getSpecies(species_id)
        model.removeSpecies(species_id)

    # Write the updated model to a new SBML file
    writer = libsbml.SBMLWriter()
    writer.writeSBMLToFile(document, output_file)
    print(f"Updated SBML file saved to {output_file}")

In [12]:
M1_original = '/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed.sbml'
M1_modified = '/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_noextras.sbml'
remove_unused_species(M1_original, M1_modified)

Updated SBML file saved to /workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_noextras.sbml


In [13]:
#remove enzymes
def remove_ecs(input_filename, output_filename):
    # Load SBML document
    document = libsbml.readSBML(input_filename)
    if document.getNumErrors() > 0:
        raise Exception('Error reading SBML file.')
    
    # Get model from document
    model = document.getModel()
    if model is None:
        raise Exception('No model found in the SBML document.')
    
    # Loop through reactions
    for i in range(model.getNumReactions()):
        reaction = model.getReaction(i)
        reactants = set(reaction.getReactant(j).getSpecies() for j in range(reaction.getNumReactants()))
        products = set(reaction.getProduct(j).getSpecies() for j in range(reaction.getNumProducts()))
        
        mutual_species = reactants & products
        
        for species in mutual_species:
            for j in range(reaction.getNumReactants()):
                if reaction.getReactant(j).getSpecies() == species:
                    reaction.removeReactant(j)
                    break  # Break after removing to avoid index issues

            for j in range(reaction.getNumProducts()):
                if reaction.getProduct(j).getSpecies() == species:
                    reaction.removeProduct(j)
                    break  # Break after removing to avoid index issues

    # Save the modified SBML document
    success = libsbml.writeSBMLToFile(document, output_filename)
    print(f"Updated SBML file saved to {output_filename}")
    if not success:
        raise Exception('Error writing SBML file.')

In [14]:
reader = libsbml.SBMLReader()
document = reader.readSBML('/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_ECfilt.sbml')
model = document.getModel()

In [15]:
def remove_ECs_2(file_in, file_out):
    # Read the SBML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(file_in)
    model = document.getModel()
    
    if model is None:
        raise ValueError("Could not find a valid model in the SBML file.")
    
    # Collect the species to remove
    species_to_remove = []
    for species in model.getListOfSpecies():
        if species.id.startswith(("e", "h")):
            species_to_remove.append(species.id)

    # Remove the collected species
    for species in species_to_remove:
        model.removeSpecies(species)

    # Write the modified SBML to a new file
    writer = libsbml.SBMLWriter()
    writer.writeSBMLToFile(document, file_out)
    
    print(f"Modified SBML saved to {file_out}")

In [16]:
M1_wECs = '/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_noextras.sbml'
M1_woutECS = '/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_ECfilt.sbml'
remove_ecs(M1_wECs, M1_woutECS)
remove_ECs_2(M1_woutECS, '/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_fin.sbml')

Updated SBML file saved to /workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_ECfilt.sbml
Modified SBML saved to /workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_fin.sbml


In [17]:
#Change this path to get new model
M1 = sbmlnetwork.load('/workspaces/ECFERS/src/frenda_brenda/Files/KEGG_Filtered/M1_renamed_fin.sbml')

### 2b. Match reaction labels (our convention) with reaction IDs (KEGG convention)

In [33]:
label_to_reaction_id

{'R10': 'R05605',
 'R11': 'R00471',
 'R62': 'R00226',
 'R63': 'R08648',
 'R105': 'R01731',
 'R113': 'R00355',
 'R123': 'R00352',
 'R157': 'R00351',
 'R171': 'R00209',
 'R175': 'R00209',
 'R181': 'R00704',
 'R204': 'R01082',
 'R227': 'R00945',
 'R228': 'R09099',
 'R268': 'R00342',
 'R270': 'R00214',
 'R271': 'R00217',
 'R272': 'R00216',
 'R273': 'R00217',
 'R283': 'R01811',
 'R307': 'R00341',
 'R308': 'R00345',
 'R346': 'R00209',
 'R347': 'R00200',
 'R348': 'R01138',
 'R349': 'R01858',
 'R350': 'R02320',
 'R351': 'R00199',
 'R378': 'R00220',
 'R385': 'R00673',
 'R406': 'R00220',
 'R197': 'R00519',
 'R269': 'R00342',
 'R345': 'R00344'}

In [31]:
M1_reactionIDs

['R21',
 'R22',
 'R23',
 'R32',
 'R50',
 'R61',
 'R71',
 'R76',
 'R77',
 'R78',
 'R79',
 'R87',
 'R110',
 'R00355',
 'R130',
 'R131',
 'R132',
 'R133',
 'R134',
 'R135',
 'R136',
 'R137',
 'R167',
 'R207',
 'R208',
 'R309',
 'R310',
 'R358',
 'R359',
 'R360',
 'R361',
 'R362',
 'R364',
 'R376',
 'R377',
 'R379',
 'R401',
 'R402',
 'R405',
 'R415',
 'R578',
 'R599',
 'R703',
 'R704',
 'R705',
 'R706',
 'R707',
 'R708',
 'R709',
 'R710']

In [18]:
M1_reaction_labels = M1.getListOfReactionIds()

df = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/Reactions_M1.csv')
label_to_reaction_id = dict(zip(df['Label'], df['Reaction ID']))
M1_reactionIDs = [label_to_reaction_id.get(item, item) for item in M1_reaction_labels]

print(f'List of reaction labels: {M1_reaction_labels}')
print()
print(f'List of corresponding reaction IDs: {M1_reactionIDs}')

label_to_ID_dict = {}
i = 0
for label in M1_reaction_labels:
    label_to_ID_dict[label] = M1_reactionIDs[i]
    i = i+1

print()
print(label_to_ID_dict)

List of reaction labels: ['R21', 'R22', 'R23', 'R32', 'R50', 'R61', 'R71', 'R76', 'R77', 'R78', 'R79', 'R87', 'R110', 'R113', 'R130', 'R131', 'R132', 'R133', 'R134', 'R135', 'R136', 'R137', 'R167', 'R207', 'R208', 'R309', 'R310', 'R358', 'R359', 'R360', 'R361', 'R362', 'R364', 'R376', 'R377', 'R379', 'R401', 'R402', 'R405', 'R415', 'R578', 'R599', 'R703', 'R704', 'R705', 'R706', 'R707', 'R708', 'R709', 'R710']

List of corresponding reaction IDs: ['R21', 'R22', 'R23', 'R32', 'R50', 'R61', 'R71', 'R76', 'R77', 'R78', 'R79', 'R87', 'R110', 'R00355', 'R130', 'R131', 'R132', 'R133', 'R134', 'R135', 'R136', 'R137', 'R167', 'R207', 'R208', 'R309', 'R310', 'R358', 'R359', 'R360', 'R361', 'R362', 'R364', 'R376', 'R377', 'R379', 'R401', 'R402', 'R405', 'R415', 'R578', 'R599', 'R703', 'R704', 'R705', 'R706', 'R707', 'R708', 'R709', 'R710']

{'R21': 'R21', 'R22': 'R22', 'R23': 'R23', 'R32': 'R32', 'R50': 'R50', 'R61': 'R61', 'R71': 'R71', 'R76': 'R76', 'R77': 'R77', 'R78': 'R78', 'R79': 'R79', 'R

In [19]:
def getSpecsinRxn(reaction):
    #get the number of species involved in the reaction
    numSpecs = M1.getNumSpeciesReferences(reaction, reaction_glyph_index=0, layout_index=0)
    specList = []
    for i in range(numSpecs):
        #get the ID of each species
        specID = M1.getSpeciesReferenceSpeciesId(reaction, reaction_glyph_index=0, species_reference_index=i, layout_index=0)
        specList.append(specID)
    return specList

In [20]:
specs_in_rxns = {}
for label in M1_reaction_labels:
    specs_in_rxns[label] = getSpecsinRxn(label)

print(specs_in_rxns)

{'R21': ['Pyr', 'LAmE', 'CO2', 'SadhlamE'], 'R22': ['Pyr', 'ThPP', 'CO2', '_2HEThPP'], 'R23': ['NAD', 'CoA', 'Pyr', 'NADH', 'CO2', 'AcCoA', 'H'], 'R32': ['NAD', 'CoA', 'Pyr', 'NADH', 'CO2', 'AcCoA', 'H'], 'R50': ['NAD', 'CoA', 'Pyr', 'NADH', 'CO2', 'AcCoA', 'H'], 'R61': ['NAD', 'Mal', 'NADH', 'Oxa', 'H'], 'R71': ['H2O', 'ATP', 'Pyr', 'PO4', 'AMP', 'PEP'], 'R76': ['ATP', 'Oxa', 'ADP', 'CO2', 'PEP'], 'R77': ['H2O', 'Glycine', '_510CH2THF', 'Serine', 'THF'], 'R78': ['Serine', 'THMPT', 'H2O', 'Glycine', '_510MTHMPT'], 'R79': ['Mal', 'H2O', 'Fum'], 'R87': ['aKG', 'LAspartate', 'Glu', 'Oxa'], 'R110': ['PO4', 'Oxa', 'H2O', 'CO2', 'PEP'], 'R113': ['PEP', 'Proteinhistidine', 'Pyr', 'ProteinNprosphosphohistidine'], 'R130': ['ATP', 'Pyr', 'ADP', 'PEP'], 'R131': ['Pyr', 'GTP', 'GDP', 'PEP'], 'R132': ['Pyr', 'CTP', 'PEP', 'CDP'], 'R133': ['Pyr', 'UTP', 'UDP', 'PEP'], 'R134': ['Pyr', 'ITP', 'PEP', 'IDP'], 'R135': ['Pyr', 'dATP', 'PEP', 'dADP'], 'R136': ['Pyr', 'dGTP', 'PEP', 'dGDP'], 'R137': ['Pyr',

### 2c. Set the coordinates in SBMLNetwork.

Save those coordinates in a list to fix autolayout locked_nodes parameter.

In [21]:
#get the graphical_object_index for species w/ alias nodes
def aliasIndexExtract(speciesID, reactionID, model):
    for i in range(model.getNumSpeciesReferences(reactionID)):
        specglyphID = model.getSpeciesReferenceSpeciesGlyphId(reactionID, species_reference_index=i)
        for j in range(model.getNumSpeciesGlyphs(speciesID)):
            objID = model.getNthSpeciesGlyphId(speciesID, j)
            if objID == specglyphID:
                return j
    return -1

### Apply a unique alias ID to every species

In [24]:
spec

'ThPP'

In [25]:
spec_react

'ThPP_R22'

In [26]:
cpd_coordinates_map_tran

{'Glycine_R00945_7890': (3160.0, 1713.0),
 '_510CH2THF_R00945_7890': (2280.0, 2169.0),
 'THF_R00945_7890': (1936.0, 2049.0),
 'Serine_R00945_7890': (2965.0, 1713.0),
 'THF_R00945_5411': (4553.0, 791.0),
 '_510CH2THF_R00945_5411': (4788.0, 792.0),
 'Serine_R09099_1212': (2965.0, 1713.0),
 '_510MTHMPT_R09099_1212': (2057.0, 2696.0),
 'Pyr_R00209_1556': (2399.0, 1713.0),
 'AcCoA_R00209_1556': (2399.0, 1834.0),
 'Oxa_R00355_2421': (2234.0, 3105.0),
 'LAspartate_R00355_2421': (2284.0, 3105.0),
 'Oxa_R00355_4044': (2321.0, 1979.0),
 'Glu_R00355_4044': (3271.0, 2333.0),
 'LAspartate_R00355_4044': (2709.0, 2013.0),
 'Pyr_R00220_4482': (2399.0, 1713.0),
 'Serine_R00220_4482': (2965.0, 1713.0),
 'PEP_R00200_3083': (2399.0, 1614.0),
 'Pyr_R00200_3083': (2399.0, 1713.0),
 'dATP_R01138_1128': (3598.0, 377.0),
 'dADP_R01138_1128': (3684.0, 377.0),
 'dGDP_R01858_1116': (4160.0, 378.0),
 'dGTP_R01858_1116': (4257.0, 378.0),
 'PEP_R02320_3083': (2399.0, 1614.0),
 'Pyr_R02320_3083': (2399.0, 1713.0),
 '

In [34]:
rxn_label

'R22'

In [22]:
M1.autolayout(max_num_connected_edges=1)
set_nodes = []
for rxn_label, specs in specs_in_rxns.items():
    corresponding_ID = label_to_ID_dict[rxn_label]
    for spec in specs:
        spec_react = spec + '_' + corresponding_ID
        index = aliasIndexExtract(spec, rxn_label, M1)

        matching_keys = [key for key in cpd_coordinates_map_tran if key.startswith(spec_react)]

        if matching_keys:
            # Add each set of coordinates to M1
            for key in matching_keys:
                coords = cpd_coordinates_map_tran[key]
                set_nodes.append(spec)
                M1.setX(spec, coords[0], graphical_object_index=index)
                M1.setY(spec, coords[1], graphical_object_index=index)
                print(f'{spec_react} in {rxn_label}/{corresponding_ID} has graphical_object_index {index} and coordinates of {coords} from KEGG')
        else:
            M1.makeSpeciesGlyphVisible(spec, rxn_label, visible=True, reaction_glyph_index=index, layout_index=0)
            print(f'{spec_react} in {rxn_label}/{corresponding_ID} has graphical_object_index {index} and no available coordinates from KEGG')
            print(M1.makeSpeciesGlyphVisible(spec, rxn_label, visible=False, reaction_glyph_index=index, layout_index=0))

Pyr_R21 in R21/R21 has graphical_object_index 0 and no available coordinates from KEGG
0
LAmE_R21 in R21/R21 has graphical_object_index 0 and no available coordinates from KEGG
0
CO2_R21 in R21/R21 has graphical_object_index 0 and no available coordinates from KEGG
0
SadhlamE_R21 in R21/R21 has graphical_object_index 0 and no available coordinates from KEGG
0
Pyr_R22 in R22/R22 has graphical_object_index 0 and no available coordinates from KEGG
0


KeyboardInterrupt: 

In [ ]:
M1.draw()

## Colorbar

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

In [ ]:
r = te.loads('/workspaces/ECFERS/src/frenda_brenda/Files/SBML models/241022_prelim_M1.sbml')

In [ ]:
r.reset()
r.simulate(0,1800)
rates = r.getReactionRates()

maxrate = rates.max()
minrate = rates.min()

In [ ]:
def get_color_from_gradient(value, min_val, max_val):
    # Normalize the value within the range
    norm = (value - min_val) / (max_val - min_val)
    # Use the color map 'coolwarm' from matplotlib
    cmap = plt.get_cmap('coolwarm')
    # Get the color in RGBA format
    rgba_color = cmap(norm)
    # Convert RGBA to HEX
    hex_color = mcolors.to_hex(rgba_color)
    return hex_color

In [ ]:
reaction_flux_dict = {reaction_id: get_color_from_gradient(rate, minrate, maxrate) for reaction_id, rate in zip(np.array(r.getReactionIds()), rates)}

In [ ]:
for rid in reaction_flux_dict.keys():
    M1.setLineColor(rid, reaction_flux_dict[rid])

In [ ]:
M1.draw()

### Hiding reactions
Now we have to figure out how to choose which reactions to hide

In [ ]:
help(M1.makeSpeciesGlyphVisible)

In [ ]:
help(M1.makeSpeciesGlyphsVisible)